In [ ]:
"""
%pip install matplotlib==3.10.1
%pip install pennylane==0.42.3
%pip install numpy==1.26.4
%pip install pandas==2.2.2
%pip install basis-set-exchange
%pip install qiskit-aer==0.17.1
%pip install pennylane-qiskit==0.40.1
%pip install qiskit-aer-gpu==0.15.1
%pip install scikit-quant==0.8.2
"""

'\n%pip install matplotlib==3.10.1\n%pip install pennylane==0.42.3\n%pip install numpy==1.26.4\n%pip install pandas==2.2.2\n%pip install basis-set-exchange\n%pip install qiskit-aer==0.17.1\n%pip install pennylane-qiskit==0.40.1\n%pip install qiskit-aer-gpu==0.15.1\n%pip install scikit-quant==0.8.2\n'

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# 1º: Create the Molecule and Hamiltonian

In the code cells below, we create the molecular geometry of H2 with the radial distance of 1,401 a.u. with the optimal internuclear distance of Szabo and Ostlund. We will use the STO-3G minimal basis set by importing the parameters of the basis set exchange functions. Pennylane allows us to construct the Hamiltonian from the second quantization with the Jordan-Wigner transformation.

In [ ]:
from qiskit_aer import AerSimulator
from skquant.opt import minimize
import matplotlib.pyplot as plt
import pennylane as qml
import pandas as pd
import numpy as np
import time
import json

symbols = ["H", "H"]
coordinates = np.array([[0.0, 0.0, 0], [0.0, 0.0, 1.401]])
molecule = qml.qchem.Molecule(symbols, coordinates, basis_name='STO-3G', load_data=True, unit='bohr')
H, qubits = qml.qchem.molecular_hamiltonian(molecule)

print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  4
The Hamiltonian is  -0.09883485860187924 * I([0, 1, 2, 3]) + 0.17120123803197834 * Z(0) + 0.17120123803197845 * Z(1) + 0.16862327620358059 * (Z(0) @ Z(1)) + -0.22279639115527738 * Z(2) + 0.12054612718324412 * (Z(0) @ Z(2)) + 0.16586801098505832 * (Z(1) @ Z(2)) + 0.045321883801814206 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.045321883801814206 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.045321883801814206 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.045321883801814206 * (X(0) @ Y(1) @ Y(2) @ X(3)) + -0.22279639115527738 * Z(3) + 0.16586801098505832 * (Z(0) @ Z(3)) + 0.12054612718324412 * (Z(1) @ Z(3)) + 0.17434948668373768 * (Z(2) @ Z(3))


# 2º: Calculate the Number of Shots Necessary for the Experiment

In the code cells below, we calculate the number of shots required for our noise-free experiment using the IMFIL optimizer. This calculation is based on sampling formulas and a specified precision, denoted by 𝜖. After determining the required number of shots, we round it to 4,000,000. Increasing the precision will result in longer computation times and may also require more memory.

In [ ]:
e = 1e-3
cj = H.terms()[0]

sum_squarred = sum([abs(float(c)) for c in cj])**2

shots_necessary = round(1/(e**2)*(sum_squarred))
shots_necessary

3935933

In [ ]:
shots_necessary=4000000

# 3º: Configure the Simulator

In the cells below, we configure a noise-free Qiskit Aer simulator using the PennyLane-Qiskit plugin. Our goal is to simulate the UCCSD ansatz with PennyLane. The quantum circuit is designed to compute the expected value of the Hamiltonian by collecting a large number of samples obtained from the shots. For these simulations, we will leverage the GPU to accelerate the computations.

In [ ]:
aer_simulator = AerSimulator(method='statevector', device='GPU')

In [ ]:
dev = qml.device("qiskit.aer", backend=aer_simulator, wires=qubits, shots=shots_necessary)

In [ ]:
electrons = 2
wires = range(qubits)
hf_state = qml.qchem.hf_state(electrons, qubits)
singles, doubles = qml.qchem.excitations(electrons, qubits)
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

In [ ]:
@qml.qnode(dev)
def circuit(params):
    qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
    return qml.expval(H)

In [ ]:
energy_data = []
params_data = []

In [ ]:
def cost_fn(param):
    energy = float(circuit(param))
    energy_data.append(energy)
    params_data.append(param)
    step = len(energy_data)
    print(f"Step {step}: Energy = {energy:.8f} Ha | θ = {param} rad")
    return energy

# 4º: Configure the optimizer and run the circuit

In the code cells below, we configure the IMFIL optimizer. It is set to a maximum of 250 iterations budget . We also provide the initial parameter angles to be optimized. The circuit is executed iteratively until convergence is achieved.

In [ ]:
init_params = np.zeros(len(singles) + len(doubles))
budget = 250
bounds = np.array([[-0.1, 0.1], [-0.1, 0.1], [-0.3, 0.3]], dtype=float)

In [ ]:
t0 = time.perf_counter()
result, history = minimize(cost_fn, init_params, bounds, budget, method='imfil')
t1 = time.perf_counter()

Step 1: Energy = -1.11668670 Ha | θ = [0. 0. 0.] rad
Step 2: Energy = -1.11483652 Ha | θ = [0.1 0.  0. ] rad
Step 3: Energy = -1.11477052 Ha | θ = [0.  0.1 0. ] rad
Step 4: Energy = -1.13498663 Ha | θ = [0.  0.  0.3] rad
Step 5: Energy = -1.11478737 Ha | θ = [-0.1  0.   0. ] rad
Step 6: Energy = -1.11478546 Ha | θ = [ 0.  -0.1  0. ] rad
Step 7: Energy = -1.02795357 Ha | θ = [ 0.   0.  -0.3] rad
Step 8: Energy = -1.12448385 Ha | θ = [ 7.33652866e-06 -2.22873481e-06  4.79243924e-02] rad
Step 9: Energy = -1.12250884 Ha | θ = [7.33652866e-06 9.99977713e-02 4.79243924e-02] rad
Step 10: Energy = -1.12247600 Ha | θ = [-9.99926635e-02 -2.22873481e-06  4.79243924e-02] rad
Step 11: Energy = -1.04659078 Ha | θ = [ 7.33652866e-06 -2.22873481e-06 -2.52075608e-01] rad
Step 12: Energy = -1.13439592 Ha | θ = [0.05 0.   0.3 ] rad
Step 13: Energy = -1.13454494 Ha | θ = [0.   0.05 0.3 ] rad
Step 14: Energy = -1.13453441 Ha | θ = [-0.05  0.    0.3 ] rad
Step 15: Energy = -1.13444458 Ha | θ = [ 0.   -0.05 

In [ ]:
opt_energy = result.optval
opt_param = result.optpar
print("\nFinal Energy Estimate:", opt_energy)
print("Optimal Parameter:", opt_energy)
print(f"Execution Time: {t1-t0}")


Final Energy Estimate: -1.1374423311150696
Optimal Parameter: -1.1374423311150696
Execution Time: 1969.075710197


# 5º: Save the Data for Later Analysis

In [ ]:
df = pd.DataFrame({
    "energies": energy_data,
    "params": params_data
})

df["energies_json"] = df["energies"].apply(lambda arr: json.dumps(arr))
df["params_json"]  = df["params"].apply(lambda arr: json.dumps(arr.tolist()))

In [ ]:
df.to_csv("IMFIL_NOISE_FREE.csv", columns=["energies_json","params_json"], index=False)